In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

## core model

In [ ]:
def CNN_model(input_features, input_shape = (512, 512, 3)):
    model = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

        keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

        keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

        keras.layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),

        keras.layers.Flatten(),

        keras.layers.Dense(1024),
        keras.layers.ReLU(),
        keras.layers.Dense(512),
        keras.layers.ReLU(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(input_features, activation='softmax')
    ])
    return model